In [2]:
!pip install torchvision
# importing
import random
import pandas as pd
import numpy as np
import torchvision
import torch
from torch import nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

## <u>Aim: To classifiy fruits images.</u>
### Dataset: Fruits360

- Then to build an iOS app to use the machine learning model.

In [22]:
# Uploading data
!git clone --recursive https://github.com/Horea94/Fruit-Images-Dataset

Cloning into 'Fruit-Images-Dataset'...
remote: Counting objects: 336735, done.
remote: Compressing objects: 100% (4656/4656), done.
remote: Total 336735 (delta 37), reused 4647 (delta 15), pack-reused 332064
Receiving objects: 100% (336735/336735), 1.58 GiB | 31.88 MiB/s, done.
Resolving deltas: 100% (716/716), done.
Checking out files: 100% (43346/43346), done.


In [0]:
# Loading data
transforms_train = T.Compose([T.ToTensor(),T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])
image_data_train = ImageFolder("./Fruit-Images-Dataset/Training",transform=transforms_train)
image_data_test = ImageFolder("./Fruit-Images-Dataset/Validation",transform=transforms_train)

# Shuffling data and then collecting all the labels.
random.shuffle(image_data_train.samples)
random.shuffle(image_data_test.samples)

In [0]:
# Total classes
classes_idx = image_data_train.class_to_idx
classes = len(image_data_train.classes)
len_train_data = len(image_data_train)
len_test_data = len(image_data_test)


def get_labels():
    labels_train = [] # All the labels
    labels_test = []
    for i in image_data_train.imgs:
        labels_train.append(i[1])
    
    for j in image_data_test.imgs:
        labels_test.append(j[1])
    
    return (labels_train, labels_test)

labels_train, labels_test = get_labels()


In [0]:
train_loader = DataLoader(dataset=image_data_train,batch_size=100)
test_loader = DataLoader(dataset=image_data_test,batch_size=100)

In [6]:
# Taken from https://stackoverflow.com/questions/48547660/attributeerror-module-pil-image-has-no-attribute-register-extensions#comment85629134_48774620
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions


print (iter(train_loader).next()[0].shape)
record = iter(train_loader).next()[0]

torch.Size([100, 3, 100, 100])


- We can see that the image is (batch_size, channel, image_height, image_width)

In [0]:
# Flatten Layer
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
# Main Model
class Model:
    def build_model(self):
        model = nn.Sequential(nn.Conv2d(3, 64, kernel_size=5, stride=1), nn.ReLU(),nn.MaxPool2d(2), nn.Conv2d(64, 64, kernel_size=7, stride=1),
                             nn.ReLU(), nn.MaxPool2d(3), nn.Conv2d(64, 64, kernel_size=7), nn.ReLU(), nn.MaxPool2d(5), Flatten(), nn.Linear(64, 100), nn.ReLU(), nn.Linear(100, 65))
        return model

In [0]:
# Building 
model = Model()
model = model.build_model()

In [9]:
print (model)
print (labels_train[0])
print (image_data_train.samples[0][1])

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=64, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=65, bias=True)
)
45
45


In [10]:
device = torch.device("cuda")
model = model.to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
torch.cuda.is_available()

True

In [0]:
# Train the model
def train(epochs):
    model.train()
    losses = []
    for epoch in range(1, epochs+1):
        print ("epoch #", epoch)
        current_loss = 0.0
        for feature, label in train_loader:
            x = Variable(feature.to(device), requires_grad=False).float()
            x = x.to(device)
            y = Variable(label.to(device), requires_grad=False).long()
            y = y.to(device)
            optimizer.zero_grad() # Zeroing the grads
            y_pred = model(x) # Calculating prediction
            correct = y_pred.max(1)[1].eq(y).sum()
            print ("no. of correct items classified: ", correct.item())
            loss = criterion(y_pred, y) # Calculating loss (log_softmax already included)
            print ("loss: ", loss.item())
            current_loss+=loss.item()
            loss.backward() # Gradient cal
            optimizer.step() # Changing weights
        losses.append(current_loss) # Only storing loss after every epoch
    return losses
# Test the model
def test():
    model.eval()
    with torch.no_grad():
        for feature, label in test_loader:
            feature = feature.to(device)
            label = label.to(device)
            pred = model(feature)
            print ("acc: ", accuracy_score(label.cpu().data.numpy(), pred.max(1)[1].cpu().data.numpy()) * 100)
            loss = criterion(pred, label)
            print ("loss: ", loss.item())

In [13]:
# Training
train(5)

epoch # 1
no. of correct items classified:  98
loss:  0.08709220588207245
no. of correct items classified:  98
loss:  0.09282206743955612
no. of correct items classified:  97
loss:  0.11759026348590851
no. of correct items classified:  98
loss:  0.08762512356042862
no. of correct items classified:  96
loss:  0.10115096718072891
no. of correct items classified:  98
loss:  0.08636155724525452
no. of correct items classified:  98
loss:  0.0757174864411354
no. of correct items classified:  95
loss:  0.14394688606262207
no. of correct items classified:  99
loss:  0.06546761840581894
no. of correct items classified:  97
loss:  0.07199545949697495
no. of correct items classified:  97
loss:  0.07301414757966995
no. of correct items classified:  98
loss:  0.07231549918651581
no. of correct items classified:  97
loss:  0.08328673988580704
no. of correct items classified:  94
loss:  0.17159534990787506
no. of correct items classified:  97
loss:  0.10689796507358551
no. of correct items classified

no. of correct items classified:  99
loss:  0.06316904723644257
no. of correct items classified:  99
loss:  0.06108799949288368
no. of correct items classified:  97
loss:  0.10739061236381531
no. of correct items classified:  99
loss:  0.06763464212417603
no. of correct items classified:  95
loss:  0.11806316673755646
no. of correct items classified:  98
loss:  0.09093305468559265
no. of correct items classified:  97
loss:  0.08653362095355988
no. of correct items classified:  94
loss:  0.11912635713815689
no. of correct items classified:  96
loss:  0.10798972845077515
no. of correct items classified:  98
loss:  0.07374796271324158
no. of correct items classified:  98
loss:  0.06464560329914093
no. of correct items classified:  99
loss:  0.04781800135970116
no. of correct items classified:  96
loss:  0.09940480440855026
no. of correct items classified:  95
loss:  0.08835490047931671
no. of correct items classified:  99
loss:  0.06387560814619064
no. of correct items classified:  97
los

no. of correct items classified:  97
loss:  0.06742651760578156
no. of correct items classified:  98
loss:  0.06716065108776093
no. of correct items classified:  98
loss:  0.07536002993583679
no. of correct items classified:  95
loss:  0.10525183379650116
no. of correct items classified:  98
loss:  0.04703483730554581
no. of correct items classified:  97
loss:  0.07468540221452713
no. of correct items classified:  97
loss:  0.09366417676210403
no. of correct items classified:  98
loss:  0.06962478905916214
no. of correct items classified:  99
loss:  0.05827423930168152
no. of correct items classified:  99
loss:  0.04856845736503601
no. of correct items classified:  97
loss:  0.0717041864991188
no. of correct items classified:  99
loss:  0.060279473662376404
no. of correct items classified:  100
loss:  0.024167470633983612
no. of correct items classified:  99
loss:  0.026502065360546112
no. of correct items classified:  98
loss:  0.06584832072257996
no. of correct items classified:  100

no. of correct items classified:  99
loss:  0.07129202038049698
no. of correct items classified:  100
loss:  0.05408865958452225
no. of correct items classified:  99
loss:  0.056500911712646484
no. of correct items classified:  96
loss:  0.08115115016698837
no. of correct items classified:  98
loss:  0.0445549376308918
no. of correct items classified:  98
loss:  0.05176655948162079
no. of correct items classified:  99
loss:  0.05844082683324814
no. of correct items classified:  97
loss:  0.08077891170978546
no. of correct items classified:  98
loss:  0.05428100749850273
no. of correct items classified:  100
loss:  0.03618887811899185
no. of correct items classified:  98
loss:  0.09411366283893585
no. of correct items classified:  98
loss:  0.06760165840387344
no. of correct items classified:  98
loss:  0.08798292279243469
no. of correct items classified:  98
loss:  0.059160731732845306
no. of correct items classified:  95
loss:  0.12391863763332367
no. of correct items classified:  99


no. of correct items classified:  100
loss:  0.04351542517542839
no. of correct items classified:  98
loss:  0.06759613752365112
no. of correct items classified:  97
loss:  0.10136869549751282
no. of correct items classified:  98
loss:  0.056357912719249725
no. of correct items classified:  97
loss:  0.09575405716896057
no. of correct items classified:  97
loss:  0.06113079935312271
no. of correct items classified:  99
loss:  0.04198862239718437
no. of correct items classified:  96
loss:  0.08780288696289062
no. of correct items classified:  97
loss:  0.11544620245695114
no. of correct items classified:  99
loss:  0.027607517316937447
no. of correct items classified:  99
loss:  0.05144725739955902
no. of correct items classified:  98
loss:  0.06100285425782204
no. of correct items classified:  98
loss:  0.06069311127066612
no. of correct items classified:  99
loss:  0.03622155264019966
no. of correct items classified:  100
loss:  0.025736674666404724
no. of correct items classified:  1

...

no. of correct items classified:  99
loss:  0.029320964589715004
no. of correct items classified:  99
loss:  0.02660154365003109
no. of correct items classified:  100
loss:  0.011347370222210884
no. of correct items classified:  100
loss:  0.012617034837603569
no. of correct items classified:  98
loss:  0.042524948716163635
no. of correct items classified:  99
loss:  0.028337139636278152
no. of correct items classified:  99
loss:  0.032037220895290375
no. of correct items classified:  100
loss:  0.012603264302015305
no. of correct items classified:  99
loss:  0.0379188247025013
no. of correct items classified:  99
loss:  0.021556777879595757
no. of correct items classified:  97
loss:  0.06380191445350647
no. of correct items classified:  99
loss:  0.030968885868787766
no. of correct items classified:  99
loss:  0.03510710597038269
no. of correct items classified:  100
loss:  0.023092404007911682
no. of correct items classified:  99
loss:  0.02338912896811962
no. of correct items classi

no. of correct items classified:  98
loss:  0.03389021009206772
no. of correct items classified:  99
loss:  0.028858881443738937
no. of correct items classified:  100
loss:  0.0062340544536709785
no. of correct items classified:  99
loss:  0.040141619741916656
no. of correct items classified:  99
loss:  0.02059650421142578
no. of correct items classified:  98
loss:  0.046006061136722565
no. of correct items classified:  99
loss:  0.030394114553928375
no. of correct items classified:  100
loss:  0.004815044347196817
no. of correct items classified:  97
loss:  0.03690178692340851
no. of correct items classified:  100
loss:  0.010695857927203178
no. of correct items classified:  100
loss:  0.01402251236140728
no. of correct items classified:  99
loss:  0.04695810377597809
no. of correct items classified:  99
loss:  0.03444956615567207
no. of correct items classified:  98
loss:  0.036785222589969635
no. of correct items classified:  100
loss:  0.020971307530999184
no. of correct items clas

no. of correct items classified:  99
loss:  0.019789524376392365
no. of correct items classified:  99
loss:  0.02777208387851715
no. of correct items classified:  100
loss:  0.016415901482105255
no. of correct items classified:  99
loss:  0.02257509157061577
no. of correct items classified:  100
loss:  0.012794227339327335
no. of correct items classified:  99
loss:  0.06133943423628807
no. of correct items classified:  99
loss:  0.020261449739336967
no. of correct items classified:  100
loss:  0.006493301596492529
no. of correct items classified:  99
loss:  0.02321542799472809
no. of correct items classified:  100
loss:  0.01563306897878647
no. of correct items classified:  100
loss:  0.011002616956830025
no. of correct items classified:  98
loss:  0.04255621135234833
no. of correct items classified:  100
loss:  0.007933635264635086
no. of correct items classified:  99
loss:  0.021164502948522568
no. of correct items classified:  100
loss:  0.007120075169950724
no. of correct items cla

no. of correct items classified:  100
loss:  0.010665607638657093
no. of correct items classified:  98
loss:  0.06093114987015724
no. of correct items classified:  98
loss:  0.027587395161390305
no. of correct items classified:  99
loss:  0.020998535677790642
no. of correct items classified:  99
loss:  0.02719554863870144
no. of correct items classified:  99
loss:  0.02284284681081772
no. of correct items classified:  100
loss:  0.004981270059943199
no. of correct items classified:  100
loss:  0.02139696106314659
no. of correct items classified:  100
loss:  0.011205101385712624
no. of correct items classified:  99
loss:  0.03219791501760483
no. of correct items classified:  100
loss:  0.019134731963276863
no. of correct items classified:  99
loss:  0.022165117785334587
no. of correct items classified:  99
loss:  0.04546424746513367
no. of correct items classified:  99
loss:  0.024835338816046715
no. of correct items classified:  100
loss:  0.015112190507352352
no. of correct items clas

[24.618884621188045,
 15.907532673329115,
 11.965506504755467,
 9.850952193140984,
 8.50047067226842]

In [0]:
# Saving model
torch.save(model.state_dict(), "model-state-dict.pth")
torch.save(optimizer.state_dict(), "optimizer-state-dict.pth")

In [15]:
!ls

datalab		      model-state-dict.pth	Training
Fruit-Images-Dataset  optimizer-state-dict.pth	Validation


In [0]:
from google.colab import files
files.download("optimizer-state-dict.pth")

In [25]:
test()

acc:  90.0
loss:  0.23974144458770752
acc:  91.0
loss:  0.31353428959846497
acc:  90.0
loss:  0.3162086606025696
acc:  91.0
loss:  0.2827865481376648
acc:  91.0
loss:  0.27352145314216614
acc:  89.0
loss:  0.27888357639312744
acc:  94.0
loss:  0.5463388562202454
acc:  91.0
loss:  0.2658222019672394
acc:  92.0
loss:  0.27742427587509155
acc:  89.0
loss:  0.3100516200065613
acc:  92.0
loss:  0.31921669840812683
acc:  88.0
loss:  0.370677649974823
acc:  90.0
loss:  0.22717849910259247
acc:  90.0
loss:  0.28181251883506775
acc:  95.0
loss:  0.11136201024055481
acc:  93.0
loss:  0.39961323142051697
acc:  88.0
loss:  0.3428434729576111
acc:  90.0
loss:  0.324552446603775
acc:  94.0
loss:  0.13432426750659943
acc:  92.0
loss:  0.25961777567863464
acc:  91.0
loss:  0.23758624494075775
acc:  92.0
loss:  0.18801970779895782
acc:  89.0
loss:  0.33798983693122864
acc:  91.0
loss:  0.3129830062389374
acc:  91.0
loss:  0.27178722620010376
acc:  94.0
loss:  0.1563756912946701
acc:  91.0
loss:  0.2538

acc:  89.0
loss:  0.3152261972427368
acc:  92.0
loss:  0.26745861768722534
acc:  88.0
loss:  0.4347172677516937
acc:  90.0
loss:  0.3350948989391327
acc:  89.0
loss:  0.5134723782539368
acc:  90.0
loss:  0.23356489837169647
acc:  97.0
loss:  0.08252861350774765
acc:  66.66666666666666
loss:  0.2955894470214844


In [26]:
# Saving complete model
torch.save(model, "complete_model.pth")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
files.download("complete_model.pth")